In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [405]:
df = pd.read_csv('./UCI_data.csv')
df.pop('date')
df.head()

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2,TARGET_energy
0,22.200000,39.500000,20.566667,37.656667,22.230000,37.030000,22.318571,36.610000,20.633333,62.166667,8.823333,13.296667,21.00,29.318571,22.23,38.463333,19.926667,33.90,9.70,766.100000,65.5,3.500000,40.000000,3.350000,24.061869,24.061869,60
1,20.356667,37.126667,17.566667,40.230000,20.890000,37.663333,18.700000,36.260000,18.463333,43.560000,1.230000,78.963333,18.39,32.290000,21.00,43.400000,18.390000,41.09,0.30,740.333333,99.0,1.000000,41.333333,0.100000,4.622052,4.622052,50
2,20.926667,38.790000,21.100000,35.526667,21.600000,36.290000,21.000000,34.826667,18.100000,46.126667,6.033333,37.300000,21.29,27.566667,21.70,32.447500,19.390000,38.76,4.40,768.466667,72.0,6.000000,22.666667,-0.266667,5.635898,5.635898,80
3,18.290000,38.900000,17.290000,39.260000,18.390000,39.326667,16.100000,38.790000,16.100000,47.700000,4.595000,94.945000,16.20,33.590000,17.78,41.663333,15.800000,39.20,3.35,760.600000,82.0,5.500000,41.000000,0.500000,49.216445,49.216445,40
4,22.290000,42.333333,21.600000,40.433333,22.666667,43.363333,19.100000,40.900000,19.290000,50.745000,2.530000,92.190000,20.00,40.000000,23.10,46.090000,18.890000,43.73,3.20,738.900000,88.0,7.333333,56.000000,1.400000,47.617579,47.617579,60


In [406]:
df.dtypes

T1               float64
RH_1             float64
T2               float64
RH_2             float64
T3               float64
RH_3             float64
T4               float64
RH_4             float64
T5               float64
RH_5             float64
T6               float64
RH_6             float64
T7               float64
RH_7             float64
T8               float64
RH_8             float64
T9               float64
RH_9             float64
T_out            float64
Press_mm_hg      float64
RH_out           float64
Windspeed        float64
Visibility       float64
Tdewpoint        float64
rv1              float64
rv2              float64
TARGET_energy      int64
dtype: object

In [407]:
df.isnull().sum()

T1               0
RH_1             0
T2               0
RH_2             0
T3               0
RH_3             0
T4               0
RH_4             0
T5               0
RH_5             0
T6               0
RH_6             0
T7               0
RH_7             0
T8               0
RH_8             0
T9               0
RH_9             0
T_out            0
Press_mm_hg      0
RH_out           0
Windspeed        0
Visibility       0
Tdewpoint        0
rv1              0
rv2              0
TARGET_energy    0
dtype: int64

In [408]:
from sklearn import preprocessing

partial_data = df[1:10001]

x = partial_data.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
complete_normalized_data = pd.DataFrame(x_scaled)

train_data= complete_normalized_data[1:4001]
test_data = complete_normalized_data[4002:8002]

print(f'Training data: {len(train_data)} Test data: {len(test_data)}')

Training data: 4000 Test data: 4000


In [409]:
target = train_data.pop(train_data.columns[len(train_data.columns)-1])
dataset = tf.data.Dataset.from_tensor_slices((train_data.values, target.values))
train_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
1,0.436818,0.356687,0.363460,0.416509,0.365570,0.341190,0.531532,0.305876,0.264692,0.244170,0.353597,0.367037,0.556079,0.154846,0.493744,0.097751,0.468262,0.399778,0.302251,0.910853,0.629139,0.428571,0.333333,0.288973,0.112652,0.112652
2,0.158395,0.360082,0.086504,0.522770,0.098870,0.489272,0.090090,0.475032,0.073579,0.268036,0.311559,0.949899,0.076343,0.368440,0.134879,0.414121,0.094693,0.418114,0.268489,0.727907,0.761589,0.392857,0.615385,0.323954,0.984688,0.984688
3,0.580781,0.466049,0.399806,0.556167,0.454193,0.686118,0.360360,0.565087,0.378404,0.314226,0.251206,0.922042,0.434496,0.595745,0.621910,0.566083,0.416233,0.606890,0.263666,0.223256,0.841060,0.523810,0.846154,0.365019,0.952695,0.952695
4,0.527983,0.231379,0.266053,0.412903,0.334829,0.288036,0.450450,0.303030,0.353878,0.126965,0.308783,0.462083,0.600377,0.108629,0.576137,0.105161,0.353798,0.459786,0.309753,0.966667,0.368653,0.166667,0.600000,0.103422,0.328664,0.328664
5,0.465681,0.335082,0.222922,0.509393,0.306580,0.499187,0.386486,0.509176,0.406965,0.617690,0.197720,0.854735,0.377003,0.392730,0.602685,0.511500,0.375650,0.496875,0.174169,0.272093,0.878587,0.130952,0.341026,0.260076,0.671153,0.671153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,0.116156,0.567901,0.072692,0.704934,0.134873,0.653446,0.149550,0.748613,0.121357,0.305959,0.431244,0.842939,0.141376,0.613475,0.063473,0.603769,0.136316,0.647312,0.434084,0.772093,0.788079,0.714286,0.600000,0.570342,0.882427,0.882427
3997,0.475185,0.561728,0.327114,0.619639,0.454655,0.697768,0.476577,0.690781,0.302914,0.341682,0.456866,0.990529,0.264844,0.629825,0.356424,0.574494,0.344433,0.692735,0.466238,0.749612,0.869757,0.535714,0.458974,0.663118,0.333483,0.333483
3998,0.866948,0.356687,0.615459,0.455598,0.672150,0.400195,0.846847,0.411012,0.729416,0.696662,0.667456,0.031325,0.791706,0.472999,0.815807,0.411740,0.780437,0.442700,0.808146,0.513953,0.408389,0.071429,0.600000,0.793916,0.852605,0.852605
3999,0.577614,0.573045,0.322268,0.681404,0.487427,0.463101,0.476577,0.694409,0.445612,0.372601,0.569414,0.338903,0.518379,0.490426,0.667684,0.480261,0.520291,0.505209,0.592712,0.480620,0.545254,0.476190,0.600000,0.623574,0.790391,0.790391


In [410]:
for f, t in dataset.take(5):
  print ('Features: {}, Target: {}'.format(f, t))

Features: [0.43681802 0.35668724 0.36346014 0.41650854 0.36556996 0.34118986
 0.53153153 0.30587566 0.26469183 0.24417047 0.35359735 0.36703741
 0.55607917 0.15484634 0.49374428 0.09775146 0.46826223 0.39977775
 0.3022508  0.91085271 0.62913907 0.42857143 0.33333333 0.28897338
 0.1126522  0.1126522 ], Target: 0.06422018348623854
Features: [0.15839493 0.3600823  0.08650351 0.5227704  0.09887006 0.48927178
 0.09009009 0.47503201 0.0735786  0.26803637 0.31155926 0.94989889
 0.07634307 0.36843972 0.13487946 0.41412061 0.09469303 0.41811363
 0.26848875 0.72790698 0.7615894  0.39285714 0.61538462 0.32395437
 0.98468809 0.98468809], Target: 0.027522935779816515
Features: [0.58078141 0.46604938 0.39980615 0.55616698 0.45419298 0.68611834
 0.36036036 0.56508749 0.3784042  0.314226   0.25120561 0.92204247
 0.43449576 0.59574468 0.62191028 0.56608308 0.41623309 0.60688985
 0.26366559 0.22325581 0.8410596  0.52380952 0.84615385 0.36501901
 0.95269518 0.95269518], Target: 0.045871559633027525
Featu

In [0]:
train_dataset = dataset.shuffle(len(train_data)).batch(1)

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation='elu'),
  tf.keras.layers.Dense(10, activation='elu'),
  tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',loss=tf.keras.losses.MeanSquaredError())

In [413]:
model.fit(train_dataset, epochs=5)

Epoch 1/5
4000/4000 [==============================] - 10s 3ms/step - loss: 0.0153
Epoch 2/5
4000/4000 [==============================] - 10s 3ms/step - loss: 0.0100
Epoch 3/5
4000/4000 [==============================] - 10s 3ms/step - loss: 0.0098
Epoch 4/5
4000/4000 [==============================] - 10s 3ms/step - loss: 0.0095
Epoch 5/5
4000/4000 [==============================] - 11s 3ms/step - loss: 0.0093


In [414]:
test_target = test_data.pop(test_data.columns[len(test_data.columns)-1])
test_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
4002,0.564238,0.372119,0.711655,0.318691,0.498504,0.406372,0.684685,0.441315,0.487020,0.198663,0.564908,0.007415,0.518379,0.429838,0.545011,0.396956,0.561915,0.494731,0.500536,0.794574,0.532009,0.404762,0.600000,0.486692,0.791449,0.791449
4003,0.538543,0.444805,0.407075,0.545541,0.298272,0.629389,0.414414,0.638782,0.428837,0.248384,0.325150,0.925784,0.387370,0.476773,0.647442,0.462696,0.386056,0.529101,0.281886,0.448837,0.927152,0.130952,0.664103,0.437262,0.749149,0.749149
4004,0.521647,0.692078,0.363460,0.743833,0.424975,0.867848,0.476577,0.804097,0.417582,0.370452,0.400068,0.989215,0.311970,0.701537,0.328044,0.689934,0.364204,0.879011,0.401929,0.625581,0.854305,0.500000,0.600000,0.565779,0.879697,0.879697
4005,0.824710,0.619033,0.486310,0.819924,0.789299,0.687906,0.881982,0.757149,0.817965,0.404557,0.614380,0.576340,0.858624,0.698582,0.887397,0.761758,0.952431,0.746909,0.630225,0.448062,0.947020,0.166667,0.394872,0.935361,0.522537,0.522537
4006,0.835269,0.643724,0.797722,0.617723,0.891825,0.753056,0.909910,0.758856,0.828476,0.298374,0.718934,0.312437,0.877474,0.656028,0.845285,0.586268,0.974402,0.716322,0.662379,0.412791,0.763797,0.202381,0.397436,0.876046,0.241156,0.241156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7997,0.465681,0.293107,0.187000,0.550878,0.415420,0.273407,0.549550,0.341300,0.398471,0.237698,0.377661,0.445568,0.377003,0.233688,0.527312,0.351070,0.485952,0.369690,0.306538,0.818605,0.810155,0.071429,0.566667,0.406084,0.870585,0.870585
7998,0.455121,0.517490,0.302399,0.649146,0.373878,0.721554,0.431532,0.688434,0.350693,0.499667,0.398217,0.956421,0.329877,0.582289,0.484590,0.823550,0.431148,0.804139,0.422294,0.218217,0.852097,0.535714,0.558974,0.590875,0.154674,0.154674
7999,0.602957,0.312654,0.420887,0.401518,0.664673,0.310631,0.666667,0.297340,0.513139,0.200231,0.479051,0.066561,0.572416,0.210402,0.676839,0.263989,0.609261,0.353313,0.448553,0.644186,0.430464,0.321429,0.600000,0.335361,0.041165,0.041165
8000,0.538543,0.252984,0.363460,0.391841,0.573280,0.249025,0.456456,0.399915,0.417582,0.338319,0.553120,0.000000,0.396795,0.242317,0.593531,0.284930,0.472078,0.380192,0.503751,0.710465,0.490066,0.416667,0.600000,0.459316,0.495892,0.495892


In [415]:
model.predict(test_data)

array([[0.09429161],
       [0.12219428],
       [0.13870357],
       ...,
       [0.11801112],
       [0.11186089],
       [0.02707902]], dtype=float32)